In [1]:
import cv2
import numpy as np
import logging
from argparse import ArgumentParser

# Local imports
import config
from utils_detection import ObjectDetector
from utils_tracking_m import ObjectTracker
from utils_visualization_m import VideoVisualizer
from analytics_roi import ROIAnalyzer
from analytics_interactions import InteractionAnalyzer

In [2]:
# Set up logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def main():
    """Main function."""
    
    # Initialize video capture
    cap = cv2.VideoCapture(config.INPUT_VIDEO)
    if not cap.isOpened():
        logger.error(f"Error opening video file: {config.INPUT_VIDEO}")
        return
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    logger.info(f"Video properties: {width}x{height} @ {fps}fps, {total_frames} frames")
    
    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    if config.WRITE_VIDEO:
        out = cv2.VideoWriter(config.OUTPUT_VIDEO, fourcc, fps // config.FRAME_SKIP, (width, height))
    
    # Initialize detector
    detector = ObjectDetector(
        model_path=config.MODEL_PATH,
        providers=config.MODEL_PROVIDERS,
        input_width=config.INPUT_WIDTH,
        input_height=config.INPUT_HEIGHT
    )
    
    # Compute diagonal for gating distance
    frame_diag = np.hypot(width, height)
    max_dist = 0.1 * frame_diag  # 10% of the diagonal
    
    # Initialize tracker
    tracker = ObjectTracker(
        iou_threshold=config.IOU_THRESHOLD,
        max_dist=max_dist,
        expire_after=config.EXPIRE_AFTER
    )
    
    # Initialize ROI analyzer
    roi_analyzer = ROIAnalyzer(config.ROIS)
    
    # Initialize interaction analyzer
    interaction_analyzer = InteractionAnalyzer(roi_analyzer.roi_diagonals)
    
    # Initialize visualizer
    visualizer = VideoVisualizer(config.COLORS)
    
    # Main loop variables
    frame_count = 0
    inoccupancy_count = 0
    
    logger.info("Starting video processing...")
    
    try:
        while cap.isOpened():
            # Read frame
            ret, frame = cap.read()
            if not ret:
                break
                
            frame_count += 1
            
            # Skip frames if needed
            if frame_count % config.FRAME_SKIP != 0:
                continue
            
            # Detect objects
            detections = detector.detect(
                frame, 
                conf_threshold=config.CONFIDENCE_THRESHOLD,
                nms_threshold=config.NMS_THRESHOLD
            )
            
            # Filter detections by ROI
            roi_filtered_boxes = roi_analyzer.filter_detections_by_roi(detections)
            
            # Update tracks
            tracked = tracker.update(
                np.array(roi_filtered_boxes),
                frame_count
            )
            
            # Calculate dwell times
            dwell_times = {
                tid: tracker.get_dwell_time(tid, frame_count, fps, config.ROIS[0])
                for tid in tracked.keys()
            }
            
            # Check ROI occupancy
            roi1_occupied = roi_analyzer.check_occupancy(tracked, roi_index=0)
            
            # Update inoccupancy counter
            if not roi1_occupied:
                inoccupancy_count += 1
            else:
                inoccupancy_count = 0
                
            # Get objects in ROIs
            roi_objects = roi_analyzer.get_objects_in_rois(tracked)
            
            # Analyze interactions
            interactions = interaction_analyzer.analyze_interactions(roi_objects, frame_count)
            
            # Visualization
            # 1. Draw tracks with dwell times
            frame = visualizer.draw_tracks(frame, tracked, dwell_times)
            
            # 2. Draw ROIs
            frame = visualizer.draw_rois(frame, config.ROIS)
            
            # 3. Draw interactions
            frame = visualizer.draw_interactions(frame, interactions, fps)
            
            # 4. Draw alert if ROI 1 is empty for too long
            if inoccupancy_count >= config.INOCCUPANCY_THRESHOLD:
                frame = visualizer.draw_alert(
                    frame, 
                    "Alert: No staff present!",
                    position=(4*width//5, 50)
                )
            
            if config.WRITE_VIDEO:
                # Write frame to output
                out.write(frame)
            
            # Display if requested
            if config.DISPLAY_VIDEO:
                cv2.imshow("Video Analytics", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                
    except KeyboardInterrupt:
        logger.info("Processing interrupted by user")
    except Exception as e:
        logger.exception(f"Error during processing: {e}")
    finally:
        # Clean up
        cap.release()
        if config.WRITE_VIDEO:
            out.release()
        cv2.destroyAllWindows()
        
        # Generate final report
        print("\n=== Final Dwell Times ===")
        final_dwell_times = tracker.get_all_dwell_times(frame_count, fps, config.ROIS[0])
        for tid, (type,seconds) in final_dwell_times.items():
            print(f"{type}-Track ID {tid}: {int(seconds)} seconds")
            
        print("\n=== Cumulative Interactions (>5s) ===")
        interactions = interaction_analyzer.get_all_interaction_durations(
            frame_count, 
            fps, 
            min_duration=config.INTERACTION_MIN_DURATION
        )
        for (id1, id2), duration in interactions.items():
            print(f"Track {id1}->{id2}: {duration:.1f}s")
            
        logger.info(f"Video processing complete. Output saved to {config.OUTPUT_VIDEO}")


In [3]:
main()

2025-05-01 19:59:25,734 - __main__ - INFO - Video properties: 1920x1080 @ 25fps, -2562047788015215 frames
/Users/saptarshimallikthakur/Desktop/tracking/.venv/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:118: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CoreMLExecutionProvider, AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
2025-05-01 19:59:25,801 - __main__ - INFO - Starting video processing...
2025-05-01 19:59:26.342 Python[41583:1039888] +[IMKClient subclass]: chose IMKClient_Modern
2025-05-01 19:59:26.342 Python[41583:1039888] +[IMKInputSession subclass]: chose IMKInputSession_Modern
[hevc @ 0x15381b810] CABAC_MAX_BIN : 7
[hevc @ 0x15381b810] The cu_qp_delta -1094995529 is outside the valid range [-26, 25].
[hevc @ 0x153b0d220] Could not find ref with POC 34
[hevc @ 0x121c05570] Could not find ref with POC 36
[hevc @ 0x121c14890] Could not find ref with POC 40
[he


=== Final Dwell Times ===
customer-Track ID 0: 13 seconds
employee-Track ID 1: 13 seconds
employee-Track ID 2: 12 seconds
employee-Track ID 3: 3 seconds

=== Cumulative Interactions (>5s) ===
